In [1]:
import tqdm
import sys
sys.path.append('data_iteration')

import numpy as np

    
from data_iterators import *
import losses

#import import_ipynb
#from results_saver import *

In [2]:
# 1. PREPARE HYPERPARAMETERS and EXPERIMENT NAME YOU WANT THE MODEL BE TRAINED WITH
model_settings = ExperimentSettings(
      name = 'testExp1'
    , hyperparameters = {'a' : 2}
    , losses = [losses.LossL1(), losses.LossL2(), losses.LossRDE_auto(), losses.LossKendall()]
    , additional_datasets = []
    , root_directory = '.test_results' #where the experiments will be saved
    , allow_skipping = True
    , allow_skipping_completed_results = True
    , save_results = True
)

In [3]:
# 2. PREPARE ITERATOR FOR ALL DATA YOU'RE INTERESTED IN
iterator_settings = RunIteratorSettings(
      zbynek_mode = False 
    , remove_duplicits_from_population = True                               
    , remove_duplicits_from_archive = True                                  
    , remove_already_known_points_from_tss_selection_process = False 
    , remove_already_known_points_from_evaluation_process = True            
    , tss = 0 # avaiable = {0,2}   
    , use_distance_weight = True                                            
)

filters = [
        lambda x: 2 == x.dimensions,
    ]
iterator = RunIterator(iterator_settings, model_settings, filters=filters)

In [4]:
# CREATE ALL files nessesary to save results
# RUN IT BEFORE PARALLEL COMPUTATION !!!

#for run in iterator:
for run in tqdm.tqdm(iterator, total=iterator.elements):
    run.initialize_experiment();

100%|██████████| 1080/1080 [00:00<00:00, 4157.09it/s]


In [5]:
import sklearn.linear_model

def experiment_run(state_iterator):
    for state in state_iterator:
        # CREATE MODEL
        lrm = sklearn.linear_model.LinearRegression(normalize=True)
        # FIT
        lrm.fit(state.x_fit, state.y_fit)
        # EVALUATE
        y_predict = lrm.predict(state.x_eval)
        # PASS RESULTS 
        losses = state.provide_results(y_predict)

        # from now losses are saved ...
        # output is only for checking
        # losses == {'L1': 4.343520880741074, 'L2': 42.579650701159586, 'RDE': 0.5714285714285714, 'Kendall': 0.032679738562091505}

In [6]:
ENABLE_MULTITHREADING = True

In [7]:
if ENABLE_MULTITHREADING:
    from multiprocessing import Pool
    N_CORES = os.cpu_count()
else:
    N_CORES = 1    
    
    
if N_CORES <= 1:
    #for run in iterator:
    for run in tqdm.tqdm(iterator, total=iterator.elements):
        for state in run:
            experiment_run(state);
else:
    with Pool(N_CORES) as p:
        v = p.imap_unordered(experiment_run, iterator)
        list(tqdm.tqdm(v, total=iterator.elements))
        

  8%|▊         | 90/1080 [00:01<00:13, 73.56it/s]


ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

In [10]:
from saver import LoaderIterator
ok_data = None
err_data = None

for val, des in iterator.get_loader():
    print(f"id={des.function_id}, di={des.dimension}, run={des.run} \t", end='')
    if val.completed:
        print('OK')
        ok_data = val
    else:
        print('Error')
        err_data = val

id=16, di=2, run=1 	Error
id=16, di=2, run=4 	Error
id=16, di=2, run=3 	Error
id=16, di=2, run=0 	Error
id=16, di=2, run=2 	Error
id=22, di=2, run=1 	Error
id=22, di=2, run=4 	Error
id=22, di=2, run=3 	Error
id=22, di=2, run=0 	Error
id=22, di=2, run=2 	Error
id=2, di=2, run=0 	OK
id=2, di=2, run=2 	OK
id=2, di=2, run=4 	OK
id=2, di=2, run=3 	OK
id=2, di=2, run=1 	OK
id=15, di=2, run=1 	Error
id=15, di=2, run=4 	Error
id=15, di=2, run=3 	Error
id=15, di=2, run=0 	Error
id=15, di=2, run=2 	Error
id=18, di=2, run=1 	Error
id=18, di=2, run=4 	Error
id=18, di=2, run=3 	Error
id=18, di=2, run=0 	Error
id=18, di=2, run=2 	Error
id=6, di=2, run=1 	Error
id=6, di=2, run=4 	Error
id=6, di=2, run=3 	Error
id=6, di=2, run=0 	Error
id=6, di=2, run=2 	Error
id=11, di=2, run=1 	Error
id=11, di=2, run=4 	Error
id=11, di=2, run=3 	Error
id=11, di=2, run=0 	Error
id=11, di=2, run=2 	Error
id=8, di=2, run=1 	Error
id=8, di=2, run=4 	Error
id=8, di=2, run=3 	Error
id=8, di=2, run=0 	Error
id=8, di=2, run

In [12]:
ok_data.data # -inf znaci prazdne hodnoty

(array([[537.4464 , 414.71014, 450.7884 , ...,      -inf,      -inf,
              -inf],
        [533.24615, 402.96893, 433.72324, ...,      -inf,      -inf,
              -inf],
        [530.1985 , 405.30865, 405.70584, ...,      -inf,      -inf,
              -inf],
        ...,
        [421.49368, 427.31067, 425.23575, ...,      -inf,      -inf,
              -inf],
        [420.3474 , 426.8352 , 427.8583 , ...,      -inf,      -inf,
              -inf],
        [429.10635, 422.65054, 426.012  , ..., 425.36462, 429.0614 ,
         425.82214]], dtype=float32),
 array([[547.3247 , 447.60547, 491.38123, ...,      -inf,      -inf,
              -inf],
        [558.7231 , 421.96658, 446.2236 , ...,      -inf,      -inf,
              -inf],
        [570.45325, 422.6085 , 422.14703, ...,      -inf,      -inf,
              -inf],
        ...,
        [422.982  , 425.3962 , 424.56686, ...,      -inf,      -inf,
              -inf],
        [424.38605, 421.69556, 422.0341 , ...,      -inf,

In [15]:
(ok_data.completed, err_data.completed)

(True, False)